# HDBSCAN should work well on embedding representations

In [4]:
import os
import re
import json
import hdbscan

import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from collections import Counter
import matplotlib.pyplot as plt

import lib.helper as helper
import lib.embedding_models as reps

from importlib import reload

%matplotlib inline

## 1.  Retrieve Corpus

The corpus is being scraped by the "run_news_scrapes.py" script (and windows task scheduler) every 12 hours, a bit past midday and a bit past midnight.

The "bing" corpus are news titles and text extracts gotten from the bing news search API, using a few Home Office - related keywords.

The "RSS" corpus is plugged directly into a number of RSS feeds for world news sites and local british news sites, with no filters for news story types or subjects applied.

In [3]:
# Should be same path for all my PC's, it's where each scrape goes as a separate json file.
storage_path = "D:/Dropbox/news_crow/scrape_results"

# "bing" is targeted news search corpus, "RSS" is from specific world and local news feeds.
corpus_type = "RSS"

# Load up
corpus = helper.load_clean_corpus(storage_path, corpus_type)

# Make sure after cleaning etc it's indexed from 0
corpus.reset_index(inplace=True)
corpus.index.name = "node"

# See how it turned out
print(corpus.shape)
corpus.head()

Total files: 325
9.8 percent of files read.
19.7 percent of files read.
29.5 percent of files read.
39.4 percent of files read.
49.2 percent of files read.
59.1 percent of files read.
68.9 percent of files read.
78.8 percent of files read.
88.6 percent of files read.
98.5 percent of files read.
(79873, 9)


,index,title,summary,date,link,source_url,retrieval_timestamp,origin,clean_text
node,,,,,,,,,
0,0,Hurricane Dorian lashes US as Bahamas counts cost,"Life-threatening US storm surges are feared, a...","Thu, 05 Sep 2019 16:03:44 GMT",https://www.bbc.co.uk/news/world-us-canada-495...,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Hurricane Dorian lashes US as Bahamas counts c...
1,1,Kohistan video murders: Three guilty in 'honou...,They are relatives of a group of Pakistani wom...,"Thu, 05 Sep 2019 13:53:17 GMT",https://www.bbc.co.uk/news/world-asia-49592540,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Kohistan video murders: Three guilty in 'honou...
2,2,MH17 Ukraine plane crash: 'Key witness' released,A Ukrainian court releases a potentially key w...,"Thu, 05 Sep 2019 13:46:06 GMT",https://www.bbc.co.uk/news/world-europe-49591148,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,MH17 Ukraine plane crash: 'Key witness' releas...
3,3,Article 370: The weddings 'ruined' by Kashmir'...,Indian-administered Kashmir is under a securit...,"Thu, 05 Sep 2019 07:32:34 GMT",https://www.bbc.co.uk/news/world-asia-india-49...,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Article 70: The weddings 'ruined' by Kashmir's...
4,4,Syria war: Turkey warns Europe of new migrant ...,President Erdogan demands international help t...,"Thu, 05 Sep 2019 16:11:48 GMT",https://www.bbc.co.uk/news/world-europe-49599297,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Syria war: Turkey warns Europe of new migrant ...


## 2.  Build Text Model (Representation, eg; word2vec, entities list...)

- Trying with the world corpus and with the bing corpus, neither worked with InferSent.  Suspect the problem lies in the PCA step, which may not be working well on this high-dimensional (vector length = 4096) form.
- Summed keywords works rather better with the world corpus.
- Summed keywords still fail the bing/home office corpus, giving me a cluster about "immigration" and a cluster for the American Supreme Court.

In [ ]:
# Windows didn't play nicely with the vector datasets, Some obscure encoding problem (python in Conda
# kept trying to decode using cp1252 regardless of whatever other options I specified!)
# Solution; rewrite file and drop any characters the Windows encoder refuses to recognise.
# I shouldn't loose too much info.
with open('./lib/InferSent/dataset/fastText/crawl-300d-2M.vec', "r", encoding="cp1252", errors="ignore") as infile:
    with open('./lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec', "wb") as outfile:
        for line in infile:
            outfile.write(line.encode('cp1252'))

In [ ]:
infersent = reps.InferSentModel(list(corpus['clean_text']),
                                list(corpus['clean_text']),
                                W2V_PATH = './lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec')

embeddings = infersent.get_embeddings()

In [ ]:
reload(reps)

In [ ]:
# Whereas this worked first time!
glove = reps.NounGloveWordModel(list(corpus['clean_text']), list(corpus['clean_text']))

embeddings = glove.get_embeddings()

In [ ]:
# Turn that into a DF for me
embeddings_df = pd.DataFrame({"clean_text": list(embeddings.keys()),
                              "embeddings": list(embeddings.values())})
embeddings_df.shape

In [ ]:
embeddings_df.head()

## 2a.  Try a really simple averaged word vector model!

With a complex noun extraction function 'cause that part's slow so I multi-threaded it.

In [6]:
from gensim.models import Word2Vec
import spacy
nlp = spacy.load('en_core_web_sm')
from gensim.models.phrases import Phrases, Phraser

In [19]:
def get_phrased_nouns(sentences):
    """ Use spacy to get all of the actual entities, conjoin bigram nouns. """

    # Get the lists of nouns
    noun_lists = []
    for doc in sentences:
        parsed = nlp(doc)
        noun_lists.append([token.lemma_ for token in parsed if token.pos_ == 'PROPN'])

    # Build the phrase model
    phrases = Phrases(noun_lists, min_count=5, threshold=0.5)

    # Get the set of phrases present in the model
    results = []
    for nouns in noun_lists:
        results.append(phrases[nouns])

    return results

# Get phrase-conjoined, lemmatized tokens
test = get_phrased_nouns(corpus['clean_text'][0:10000])

# Detect and conjoin bigrams
model = Word2Vec(test, size=100, window=5, min_count=1, workers=10)

In [62]:
def get_averaged_vec(token_list, model):
    
    vecs = []
    for token in token_list:
        try:
            vector = model.wv[token]
        except: 
            vector = np.zeros(100)
        vecs.append(vector)
    
    if len(vecs) > 0:
        return np.mean(np.asarray(vecs), axis=0)
    else:
        return np.zeros(100)

In [63]:
vectors = [get_averaged_vec(tokens, model) for tokens in test]

In [67]:
vectors[0:2]

[array([ 5.3463317e-03,  2.5220399e-03, -4.6524671e-03,  3.4887677e-03,
         8.2033938e-03, -4.9915398e-03, -8.0874393e-04,  1.8334268e-03,
         2.5391961e-03, -2.2249611e-03, -3.1220273e-03,  1.7007247e-04,
         2.2594405e-03, -3.5947412e-03,  1.4751510e-03,  1.8720523e-03,
        -1.4218759e-03, -1.0979190e-03, -3.1866290e-05,  7.8581972e-04,
         2.2808164e-03,  5.5785262e-04, -2.0109962e-03,  4.2855716e-03,
        -1.6455781e-03, -6.6904543e-04,  1.2567308e-03,  2.8988707e-03,
        -1.5578925e-03,  3.7417940e-03,  1.3559783e-03,  4.6948856e-04,
        -1.9674024e-03,  1.6709786e-03,  2.9800681e-03,  4.1622436e-03,
         4.9548876e-03,  3.3579834e-03,  3.5455704e-04, -4.6961778e-03,
        -2.1982871e-03, -2.5425551e-03,  2.2553939e-03, -3.8203963e-03,
        -2.0974192e-03,  6.3258543e-04,  7.0130778e-04, -5.7315041e-04,
         6.3947425e-03,  6.9253738e-03,  2.3834405e-03,  1.6529089e-03,
         4.8593953e-04,  9.5911942e-05, -3.9122058e-03,  4.02796

## 3. Cluster Text

This is the part where the pipelines get a little more experimental

In [68]:
embeddings_array = np.vstack(vectors)
#embeddings_array = np.vstack(embeddings_df['embeddings'])

In [69]:
embeddings_array.shape

(10000, 100)

In [70]:
# First, PCA the data
pca = PCA(n_components=20, svd_solver='full')

embeddings_pca = pca.fit_transform(embeddings_array)

In [71]:
embeddings_pca.shape

(10000, 20)

In [72]:
print(pca.explained_variance_ratio_)
print(pca.singular_values_) 

[0.37893687 0.02608303 0.02410494 0.01998329 0.01535319 0.01497381
 0.01334177 0.01224024 0.01178881 0.01097897 0.01063203 0.01043303
 0.01002969 0.00953764 0.00937127 0.00906266 0.0088502  0.0085055
 0.00822424 0.00807814]
[1.39724507 0.3665795  0.35240507 0.3208652  0.28124736 0.27775079
 0.26217776 0.2511216  0.24644738 0.23783183 0.23404385 0.23184325
 0.22731753 0.22167133 0.21972947 0.21608122 0.21353336 0.20933361
 0.20584339 0.20400684]


In [73]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=1)
clusterer.fit(embeddings_array)
pd.unique(clusterer.labels_)

array([ -1, 128,  19, 116,  71, 100,   4,  76,   3,   2,  18,  39,   6,
        65,  78,  48,  32,   5,  50,  77,  38,  57,  51,  94, 104,  22,
         9,  69,  17,  75, 118,  20,  31, 101, 109, 119,  98,  97, 107,
       103,  99, 110,  93,  49,  45,  15,  72,  74,  52, 106,  87,  44,
       129, 111,  59,  84,  29,  43,  53,  34,  40,  16,  36, 115, 117,
        61, 130,  26,  60,   8,  23,  25,  73,  41, 102, 112,  42,   7,
        62,  55, 108,  47, 105,  67,  89,  10,  14,  27, 126, 125,  96,
        79,  88,  12,  13,  63, 124,  64,  30,  95,  33,  66, 114,  21,
        70,  83,  28,  92,  85, 127,  81, 113,  68,  46,  56,  35,  90,
       122,  37,  24,  80, 121,   1,  54, 123,  11,  82,   0,  91,  58,
        86, 120], dtype=int64)

In [74]:
len(pd.unique(clusterer.labels_))

132

In [75]:
Counter(clusterer.labels_)

Counter({-1: 8328,
         128: 628,
         19: 8,
         116: 8,
         71: 5,
         100: 5,
         4: 30,
         76: 20,
         3: 8,
         2: 7,
         18: 6,
         39: 5,
         6: 6,
         65: 6,
         78: 5,
         48: 20,
         32: 5,
         5: 15,
         50: 5,
         77: 7,
         38: 6,
         57: 7,
         51: 6,
         94: 6,
         104: 5,
         22: 6,
         9: 8,
         69: 9,
         17: 6,
         75: 5,
         118: 5,
         20: 5,
         31: 5,
         101: 6,
         109: 10,
         119: 5,
         98: 7,
         97: 7,
         107: 5,
         103: 9,
         99: 6,
         110: 5,
         93: 8,
         49: 7,
         45: 5,
         15: 19,
         72: 8,
         74: 9,
         52: 6,
         106: 5,
         87: 6,
         44: 8,
         129: 27,
         111: 20,
         59: 5,
         84: 6,
         29: 7,
         43: 6,
         53: 39,
         34: 14,
         40: 22,
